In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd
import seaborn as sns

In [ ]:
start = time.time()

# At Siena
#topdir = '/mnt/qnap/babar_data/bnv_plambda'

##josie laptop 
#topdir = "/Users/josieswann/Desktop/important documents"

# On Bellis' laptop
#topdir = '/home/bellis/babar_data/bnv_plambda/'

# At Bellis' home
topdir = '/home/bellis/babar_data/bnv_plambda'

# On Bellis' laptop
#topdir = './'

filename = f'{topdir}/Background_and_signal_SP_modes_Only_Run_1.parquet'
#filename = f'{topdir}/Background_and_signal_SP_modes_All_runs.parquet'

data = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")
IS_MC=True


#'''
# Collision data
#filename = f'{topdir}/Background_SP_modes_Only_Run_1.parquet'
filename = f'{topdir}/Data_Only_Run_1_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'
#filename = f'{topdir}/Data_All_runs_BLINDED.parquet'

start = time.time()

data_collision = ak.from_parquet(filename)
#data_collision = ak.from_parquet(filename)

print(f"Took {time.time() - start} s")

print(type(data_collision))
#'''


In [ ]:
### information about cross section --> what we'll use to calculate scaling values for histograms 

dataset_information = pd.read_csv("dataset_statistics.csv")
cs_data= pd.read_csv("SP_cross_sections_and_labels.csv")

no_notes= cs_data.drop(["Uncertainty","Note: cross sections found at https://babar-wiki.heprc.uvic.ca/bbr_wiki/index.php/Physics/Cross_sections,_luminosities,_and_other_vital_stats"], axis= 1)
no_notes

In [ ]:
sp = data['spmode']

np.unique(sp.to_list())

In [ ]:
region_definitions

In [ ]:
#bat.fill_new_entry_with_tag_side_B(data)
#data['BtagSideMes']



In [ ]:
# Make our histograms
all_hists = bat.create_empty_histograms(hist_defs)

bkg_spmodes = ['998', '1005', '1235', '1237', '3981']
sig_spmodes = ['-999']

spmodes = bkg_spmodes + sig_spmodes

weights = {}
for sp in spmodes:
    weights[sp] = bat.scaling_value(int(sp), dataset_information=dataset_information, cs_data=cs_data, plot=False, verbose=False)
    #weights[sp] = 1

### bat.scaling_value is in Babar_analysis_tools.py 

# Scale the signal higher
weights['-999'] = 1000
weights['0'] = 1

print(weights)
print()
print(spmodes)

In [ ]:
# Need to get the original duplicates mask for any other cuts we might generate outside the function
dcuts = bat.get_final_masks(data, region_definitions=region_definitions)

print([dcuts.keys()])
print()

for key in dcuts.keys():
    print(f'{key:3d} {dcuts[key]["name"]}')



In [ ]:
###############################################################################
# Make the masks
#mask_event = dcuts[3]['event']
#mask_event = dcuts[4]['event']
mask_event = dcuts[1]['event']
#mask_event = dcuts[-1]['event']
#mask_event = dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event']

spmode = '998'

mask_sp = data['spmode']==spmode

#mask = mask_event & mask_sp
mask = mask_sp

# Make the plot
mes =    ak.flatten(data[mask]['BpostFitMes'])
DeltaE = ak.flatten(data[mask]['BpostFitDeltaE'])

bat.plot_mes_vs_DeltaE(mes, DeltaE, region_definitions=region_definitions, draw_signal_region=True)

#print(len(mes),len(DeltaE))

## Other HISTOGRAMS 
all_hists = bat.create_empty_histograms(hist_defs)

x = ak.flatten(data[mask]['Lambda0_unc_Mass'])
weight = weights[spmode]
all_hists['Lambda0_unc_Mass'].fill(var=x, SP= spmode, cuts= f"{0}", weight= weight)

plt.figure()
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="fill", color='red', label= spmode)
all_hists['Lambda0_unc_Mass'].project('var').plot(histtype="step", color='black')
plt.legend();

# pyhf

https://pyhf.github.io/pyhf-tutorial/HelloWorld.html

https://pyhf.github.io/pyhf-tutorial/IntroToHiFa.html


## A one-bin example

https://pyhf.readthedocs.io/en/v0.7.6/

## Dummy 2D thing

https://pyhf.readthedocs.io/en/v0.7.6/examples/notebooks/multiBinPois.html

## 2D distributions

https://stackoverflow.com/questions/66038864/2d-distributions-in-the-histfactory

Maybe this thread?


https://github.com/scikit-hep/pyhf/discussions/2070

Use `ravel`

Drawing on some work I did here

https://colab.research.google.com/drive/1J4IQ8DDkKKQPxQH2A06l1Z350YlWqTL7?usp=sharing

In [ ]:
import pyhf

import json

import matplotlib.pyplot as plt
import numpy as np
from pyhf.contrib.viz import brazil

import logging
import json
import math
import numpy as np
import matplotlib.pyplot as plt

import pyhf
from pyhf import Model, optimizer
from pyhf.simplemodels import uncorrelated_background
from pyhf.contrib.viz import brazil

from scipy.interpolate import griddata
import scrapbook as sb



In [ ]:
#https://pyhf.github.io/pyhf-tutorial/IntroToHiFa.html

def draw_plot_dm(mu=1):
    bins = [5, 55, 550]
    observed = [60, 43, 40]
    background = [58.0, 42.5, 36.0]
    signal = [i * mu for i in [2, 4, 3]]

    print(f"signal: {signal}")
    print(f"background: {background}")
    print(f"observed: {observed}\n")

    fig, ax = plt.subplots()
    ax.bar(bins, background, width=[10.0, 90, 900], label=r"Background", edgecolor="red", alpha=0.5)
    ax.bar(
        bins,
        signal,
        width=[10.0, 90, 900],
        label=r"Signal",
        edgecolor="blue",
        bottom=background,
        alpha=0.5,
    )
    ax.scatter(bins, observed, color="black", label="Observed")
    #ax.set_ylim(0, 6)
    ax.legend()
    ax.set_xlabel(r"$p_T$ [GeV/c]", fontsize=12)
    ax.set_ylabel("Count", fontsize=12);


draw_plot_dm()

In [ ]:
#https://pyhf.github.io/pyhf-tutorial/IntroToHiFa.html

def draw_plot_dm(mu=1):
    bins = [5, 55, 550]
    observed = [60, 43, 40]
    background = [58.0, 42.5, 36.0]
    signal = [i * mu for i in [2, 4, 3]]

    print(f"signal: {signal}")
    print(f"background: {background}")
    print(f"observed: {observed}\n")

    fig, ax = plt.subplots()
    ax.bar(bins, background, width=[10.0, 90, 900], label=r"Background", edgecolor="red", alpha=0.5)
    ax.bar(
        bins,
        signal,
        width=[10.0, 90, 900],
        label=r"Signal",
        edgecolor="blue",
        bottom=background,
        alpha=0.5,
    )
    ax.scatter(bins, observed, color="black", label="Observed")
    #ax.set_ylim(0, 6)
    ax.legend()
    ax.set_xlabel(r"$p_T$ [GeV/c]", fontsize=12)
    ax.set_ylabel("Count", fontsize=12);


draw_plot_dm()

In [ ]:
# Get the data

# Make the masks
#mask_event = dcuts[3]['event']
#mask_event = dcuts[4]['event']
mask_event = dcuts[1]['event']
#mask_event = dcuts[-1]['event']
#mask_event = dcuts[2]['event'] & dcuts[3]['event'] & dcuts[4]['event']

spmode = '998'

mask_sp = data['spmode']==spmode

#mask = mask_event & mask_sp
mask = mask_sp

# Make the plot
mes =    ak.flatten(data[mask]['BpostFitMes'])
DeltaE = ak.flatten(data[mask]['BpostFitDeltaE'])

n = 100
mes = np.random.choice(mes.to_numpy(), n)
DeltaE = np.random.choice(DeltaE.to_numpy(), n)

print(len(mes), mes)
print(len(DeltaE), DeltaE)

In [ ]:
plt.figure()
plt.plot(mes, DeltaE, '.')
plt.xlabel('mES')
plt.ylabel('DeltaE')
;

In [ ]:
vals, xedges, yedges = np.histogram2d(mes, DeltaE, bins=[5, 5], range=[[5.2, 5.3], [-0.2, 0.2]])
print(vals)

vals = np.ravel(vals).tolist()
print(vals)


In [ ]:
# 2D

sig = np.ravel([[12.0, 11.0], [5, 6]]).tolist()
bkg = np.ravel([[50.0, 52.0],[7, 7]]).tolist()
bkg_uncert = np.ravel([[3.0, 7.0], [1, 2]]).tolist()

print(sig)
print(bkg)
print(bkg_uncert)



'''

model = pyhf.simplemodels.uncorrelated_background(
    signal=sig, bkg=bkg, bkg_uncertainty=bkg_uncert
)

print("aux data")
print(model.config.auxdata)

data = np.ravel([[51, 48], [8, 4]]).tolist() + model.config.auxdata

test_mu = 1.0

print("model")
print(model)
print()

print("data")

print(data)

CLs_obs, CLs_exp = pyhf.infer.hypotest(
    test_mu, data, model, test_stat="qtilde", return_expected=True
)

print(f"Observed: {CLs_obs}, Expected: {CLs_exp}")

'''